In [3]:
### step1
# Imports
###
import csv
import json
import os
import pprint
from dictor import dictor
import pandas as pd
from copy import copy
import requests

In [4]:
UpdatequeryByTripCode = """
mutation UpdatequeryByTripCode($tripCode: String!, $countryId: uuid) {
  update_trip(where: {_and: [{code: {_eq: $tripCode}}]}, _set: {main_country_id: $countryId}) {
    affected_rows
  }
}
"""
UpdatequeryByBrandCode = """
mutation updateTripMainCountry($brandCode: String!, $countryId: uuid) {
  update_trip(where: {_and: [{project: {code: {_eq: $brandCode}}}]}, _set: {main_country_id: $countryId}) {
    affected_rows
  }
}
"""

In [5]:
### step4
# Declare url and authentication with header secret FIXME protect this
###
headers = {'content-type': 'application/json', 'x-hasura-admin-secret': 'V3ntur4Pr0dH4sur4Tr4v3lC0re2o2o'}
url = 'https://core.venturatravel.org/v1/graphql'

In [6]:
def sendRequest(query, variables: dict={}):
    r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
    return r.json()


In [30]:
associations  = [
    { 'tripCode': 'carg', 'countryName': 'Argentina' },
    { 'tripCode': 'cpat', 'countryName': 'Argentina' },
    { 'tripCode': 'cpatcp', 'countryName': 'Argentina' },
    { 'tripCode': 'cchin', 'countryName': 'China' },
    { 'tripCode': 'xchip', 'countryName': 'China' },
    { 'tripCode': 'xchinm', 'countryName': 'China' },
    { 'tripCode': 'ccol', 'countryName': 'Colombia' },
    { 'tripCode': 'xpsujg', 'countryName': 'Peru' },
    { 'tripCode': 'xpsucp', 'countryName': 'Peru' },
    { 'tripCode': 'ysriMIA', 'countryName': 'Sri Lanka' },
    { 'tripCode': 'yprtMIA', 'countryName': 'Portugal' },
    { 'tripCode': 'xgrex', 'countryName': 'Greece' },
    { 'tripCode': 'xegacp', 'countryName': 'Ecuador' }, 
]


In [19]:
# get all countries
queryCountries = """
query getCountries{
    country{
        id
        code
        name
    }
}
"""
res = sendRequest(queryCountries)
countries = res['data']['country']

In [24]:
def findCountryId(countries, countryCode):
    return next(x['id'] for x in countries if x['code'] == countryCode)

def findCountryIdByName(countries, countryName):
    return next(x['id'] for x in countries if x['name'] == countryName)

In [31]:
updateByTripCode = []
updateByBrandCode = []
noFoundCountry = []
for assoc in associations:
    countryId = None
    if 'countryCode' in assoc:
        countryId = findCountryId(countries,  assoc['countryCode'])
    elif 'countryName' in assoc:
        countryId = findCountryIdByName(countries,  assoc['countryName'])
        
    if countryId == None:
        noFoundCountry.append(assoc)
        continue
    
    if dictor(assoc, 'brandCode') != None:
        updateByBrandCode.append({ 'brandCode': dictor(assoc, 'brandCode'), 'countryId': countryId })
    elif dictor(assoc, 'tripCode') != None:
        updateByTripCode.append({ 'tripCode': dictor(assoc, 'tripCode'), 'countryId': countryId })
                              

if len(noFoundCountry) > 0:
    print('Countries not found')
    print(noFoundCountry)


In [16]:
for obj in updateByBrandCode:
    print('___________')
    print(obj)
    resUpdate = sendRequest(UpdatequeryByBrandCode, obj)
    print(resUpdate)

___________
{'brandCode': 'eur', 'countryId': 'cedbeeb9-3133-4048-b56b-4cddaf2eac22'}
{'data': {'update_trip': {'affected_rows': 15}}}
___________
{'brandCode': 'fai', 'countryId': '00d5bca0-7cf6-4775-b95f-25e5bb41e411'}
{'data': {'update_trip': {'affected_rows': 20}}}


In [32]:
for obj in updateByTripCode:
    print('___________')
    print(obj)
    resUpdate = sendRequest(UpdatequeryByTripCode, obj)
    print(resUpdate)

___________
{'tripCode': 'carg', 'countryId': '3b6f539a-7402-4518-b606-d4053fb0b6a9'}
{'data': {'update_trip': {'affected_rows': 5}}}
___________
{'tripCode': 'cpat', 'countryId': '3b6f539a-7402-4518-b606-d4053fb0b6a9'}
{'data': {'update_trip': {'affected_rows': 4}}}
___________
{'tripCode': 'cpatcp', 'countryId': '3b6f539a-7402-4518-b606-d4053fb0b6a9'}
{'data': {'update_trip': {'affected_rows': 3}}}
___________
{'tripCode': 'cchin', 'countryId': '02664588-3a2d-4cde-b047-e117bce8ac4a'}
{'data': {'update_trip': {'affected_rows': 1}}}
___________
{'tripCode': 'xchip', 'countryId': '02664588-3a2d-4cde-b047-e117bce8ac4a'}
{'data': {'update_trip': {'affected_rows': 1}}}
___________
{'tripCode': 'xchinm', 'countryId': '02664588-3a2d-4cde-b047-e117bce8ac4a'}
{'data': {'update_trip': {'affected_rows': 1}}}
___________
{'tripCode': 'ccol', 'countryId': 'fe73a8cc-4e10-4e49-a429-898e13912d84'}
{'data': {'update_trip': {'affected_rows': 4}}}
___________
{'tripCode': 'xpsujg', 'countryId': 'cbecc43